In [3]:
from scipy.spatial.transform import Rotation as R # Magnitude (and '*_matrix', use '*_dcm' instead) require a newer version of SciPy than SPIN's
import numpy as np

In [2]:
# r is a rotation matrix that doesn't rotate anything

r = R.from_matrix([[1,0,0],
                 [0,1,0],
                 [0,0,1]])

AttributeError: type object 'Rotation' has no attribute 'from_matrix'

In [ ]:
r.magnitude()

In [ ]:
# q is a rotation matrix that rotates things 90 degrees around the z axis
# we know this both intuitively as well as by applying the basic rotation formula
# https://en.wikipedia.org/wiki/Rotation_matrix#Basic_rotations
# (cosine of 90 is 0, sine of 90 is 1)

q = R.from_matrix([[0,-1,0],
                  [1,0,0],
                  [0,0,1]])

In [ ]:
np.degrees((r*q).magnitude())

In [ ]:
# m is a rotation matrix that rotates 45 degrees around the z axis

m = R.from_matrix([[np.cos(np.pi/4),-np.sin(np.pi/4),0],
                  [np.sin(np.pi/4),np.cos(np.pi/4),0],
                  [0,0,1]])

In [ ]:
np.degrees(m.magnitude())

In [ ]:
# you can tell composing these rotations isn't very impressive - is like having just one
# 135 degree rotation

np.degrees((m*q).magnitude())

In [ ]:
# now what if we have a more complex rotation, composing with of 30 degree along the x axis

n = R.from_matrix([[1,0,0],
                  [0,np.cos(np.pi/6),-np.sin(np.pi/6)],
                  [0,np.sin(np.pi/6),np.cos(np.pi/6)]])

In [ ]:
np.degrees(n.magnitude())

In [ ]:
# we can tell the magnitude doesn't tell us much about which part is played by each
# axis on the rotation

np.degrees((q*n).magnitude())

In [ ]:
# however the euler representation makes it clear
# (for some reason 'zyx' seems to be the default)

(q*n).as_euler('zyx', degrees=True)

# notice what this means: q*n is a rotation that takes a vector, rotates it 30 degrees
# around x and then 90 degrees around z

In [ ]:
(n*q).as_euler('xyz', degrees=True)

# Interestingly, rotating frames appear to be different than rotating vectors, see
# the wikipedia page

In [ ]:
(q).as_matrix()

In [ ]:
(q*n).as_matrix()

In [16]:
# Let's test this by loading up a real example from 3dpw

import pickle as pkl

img_path = 'examples/image_00502_crop.jpg'
#img_path = 'examples/image_00980.jpg'

pickle_path = 'data/3dpw/sequenceFiles/validation/courtyard_basketball_01.pkl'
#pickle_path = 'data/3dpw/sequenceFiles/validation/outdoors_parcours_01.pkl'

frame = 502

In [10]:
# using IPython's Magic %run let's us cleanly run the Demo script and get the variables defined
# therein into this notebook. use $ to insert variables from the notebook into any %magic command
%run demo.py --checkpoint=data/model_checkpoint.pt --img=$img_path

In [35]:
# open the sequence file, fetch the body_pose of the corresponding frame from it, remove global orientation and
# input into a rotation object

seq = pkl.load(open(pickle_path,'rb'),encoding='latin-1')
gt_pose_axis_angle = seq['poses'][0][frame][3:]
gt_pose = R.from_rotvec(np.reshape(gt_pose_axis_angle, (23,-1)))

In [46]:
# create a rotation object from the predicted pose output of demo.py

pred_pose = R.from_dcm(pred_output.body_pose.squeeze().cpu())

In [49]:
# show the difference between the predicted and the ground truth pose

R.as_euler(pred_pose*gt_pose, 'xyz', degrees=True)

array([[ -11.53423152,    1.45218796,   14.39380192],
       [  25.35587121,   12.15640678,  -13.92902663],
       [  31.62514251,    1.84057256,    6.65371351],
       [  61.65764612,   -6.91880428,  -10.57649552],
       [  76.14436648,   11.0574346 ,   12.02639854],
       [  -2.9572873 ,    3.88892113,   -4.60218202],
       [  -8.42285578,    9.52938331,  -14.11790154],
       [  -2.02860379,   -4.93019691,    5.4100804 ],
       [  -4.14222322,    4.97110844,   -2.28494807],
       [ -10.33501961,    6.29445553,   15.34893562],
       [ -10.95823116,    2.31976112,  -10.11235415],
       [ -32.3080475 ,   23.59630993,  -37.96448256],
       [ -57.62417828,  -40.64518083,   44.1261785 ],
       [ -39.54596354,   33.30874471,  -28.71964085],
       [   6.93876137,   -0.69058056,   20.21904431],
       [ -81.56851515,  -69.93777261,   35.42649026],
       [-159.17950351,   51.76062816, -117.68050672],
       [ -77.29650741,    2.80020533,  141.25024586],
       [-134.7210253 ,   12.

In [37]:
# Now, let's check through examples that this behavior makes sense

array([[  3.27758532,   2.34568285,  10.05848473],
       [ 18.07793628,  15.48206547,  -8.55389247],
       [ 25.68959525,  -1.39709914,   5.26220588],
       [ 27.52093489,  -2.67720552,  -2.25343333],
       [ 33.83171014,   6.25581228,   3.7944147 ],
       [ -0.5386765 ,   5.39381936,  -5.35365826],
       [ -4.95139093,   1.41181823,  -6.09309659],
       [  7.59999533,   7.79373372,  -4.2199807 ],
       [ -3.00579048,   3.15721984,  -3.11590527],
       [ -0.57181028,  -0.65411145,   2.92180118],
       [ -2.48339326,  -5.21716679,   3.6632402 ],
       [-17.19975338,  19.26979555, -33.02213953],
       [-21.11404173, -19.81535274,  16.22398535],
       [-10.74261673,   4.45868369, -10.92492939],
       [ -2.62982222,  -2.43141453,  24.53937954],
       [-30.34483409, -37.38967198,  -6.87046638],
       [-70.95835327,  66.67429707, -43.30055846],
       [ -7.99851588, -45.37535478,  75.74406625],
       [-37.67749076,  67.31087175, -44.69764436],
       [ 21.33930871, -12.18248